<a href="https://colab.research.google.com/github/vbonato/cnnTestBench/blob/main/pauNaJaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision





---


---



---
**Dataset WISDM**



---



---



---





In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Create time windows

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
import re

# Function to create time windows
def create_time_windows(data, labels, window_size):
    X = []
    y = []

    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])  # Select window of data
        y.append(labels[i + (window_size-1)])  # Label is from the last element of the window

    return np.array(X), np.array(y)

# Function to save windowed data for your C++ program
def save_windowed_data_to_dat(X, y, file_path):
    """Saves the windowed data to a .dat file."""
    with open(file_path, 'w') as f:
        for i in range(X.shape[0]):
            feature_vector = X[i].flatten()
            feature_str = ",".join(map(str, feature_vector))
            label = y[i]
            f.write(f"{{{{{feature_str}}},{label}}}\n")
    print(f"Successfully saved {X.shape[0]} samples to {file_path}")

# Updated data loading function
def load_data(file_path):
    features = []
    labels = []

    with open(file_path, 'r') as f:
        for line in f:
            # Use regular expression to match the pattern {{feature_vector}, label}
            match = re.match(r"\{\{([0-9.,-]+)\},\s*(\d+)\}", line.strip())

            if match:
                # Extract feature vector and label
                feature_str = match.group(1)  # The feature string "8.24,-2.11,3.87"
                label = int(match.group(2))  # The label "4"

                # Convert the feature string to a list of floats
                feature_vector = list(map(float, feature_str.split(',')))

                features.append(feature_vector)
                labels.append(label)

    return np.array(features), np.array(labels)

# Load train and test data
train_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/WISDM/HAR-Dataset/train.dat'  # Adjust path to your file
test_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/WISDM/HAR-Dataset/test.dat'  # Adjust path to your file

# Define the window size
window_size = 10

# Create time windows
X_train, y_train = load_data(train_data_file)
X_train, y_train = create_time_windows(X_train, y_train, window_size)

X_test, y_test = load_data(test_data_file)
X_test, y_test = create_time_windows(X_test, y_test, window_size)

# --- 2. START: SAVE DATA FOR C++ TRAINER ---
# Define where you want to save the new files
output_path_train = 'train_windowed.dat' # You can change this path
output_path_test = 'test_windowed.dat'   # You can change this path

# Call the save function
save_windowed_data_to_dat(X_train, y_train, output_path_train)
save_windowed_data_to_dat(X_test, y_test, output_path_test)
# --- END: SAVE DATA FOR C++ TRAINER ---

# Print the first feature vector and label
print("First feature vector in X_train:")
print(X_train[0])  # First row (first feature vector)
print("First label in y_train:")
print(y_train[0])  # First label

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # For classification (long type for labels)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Print the first feature vector and label
print("First feature vector in X_train:")
print(X_train[0])  # First row (first feature vector)
print("First label in y_train:")
print(y_train[0])  # First label

# Check the shapes of the loaded data
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

Successfully saved 738432 samples to train_windowed.dat
Successfully saved 316462 samples to test_windowed.dat
First feature vector in X_train:
[[-0.6946377  12.680544    0.50395286]
 [ 5.012288   11.264028    0.95342433]
 [ 4.903325   10.882658   -0.08172209]
 [-0.61291564 18.496431    3.0237172 ]
 [-1.1849703  12.108489    7.205164  ]
 [ 1.3756552  -2.4925237  -6.510526  ]
 [-0.61291564 10.56939     5.706926  ]
 [-0.50395286 13.947236    7.0553403 ]
 [-8.430995   11.413852    5.134871  ]
 [ 0.95342433  1.3756552   1.6480621 ]]
First label in y_train:
0
First feature vector in X_train:
tensor([[-0.6946, 12.6805,  0.5040],
        [ 5.0123, 11.2640,  0.9534],
        [ 4.9033, 10.8827, -0.0817],
        [-0.6129, 18.4964,  3.0237],
        [-1.1850, 12.1085,  7.2052],
        [ 1.3757, -2.4925, -6.5105],
        [-0.6129, 10.5694,  5.7069],
        [-0.5040, 13.9472,  7.0553],
        [-8.4310, 11.4139,  5.1349],
        [ 0.9534,  1.3757,  1.6481]])
First label in y_train:
tensor(0)
X

In [4]:
# Define a model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # CNN layers
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, padding=1)  # input channels = number of features
        #self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, padding=1)  # input channels = number of features

        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        # Fully connected layers
        # Calculate the input size for the fully connected layer based on the output size of conv1
        self.fc1 = nn.Linear(16 * (X_train.shape[1] // 2), 64)  # Flattened size after pooling
        self.fc2 = nn.Linear(64, 6)  # Assuming 6 classes for classification

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Add channel dimension (change shape to [batch_size, channels, seq_length])

        #x = torch.relu(self.conv1(x))  # Apply conv1 and pooling
        #x = self.pool(torch.relu(self.conv2(x)))  # Apply conv2 and pooling
        x = self.pool(torch.relu(self.conv1(x)))  # Apply conv1 and pooling

        x = x.view(-1, 16 * (x.shape[2]))  # Flatten for fully connected layer
        x = torch.relu(self.fc1(x))  # Apply first fully connected layer
        x = self.fc2(x)  # Output layer (no activation since we'll apply softmax in loss)
        return x


# Initialize the model
model = SimpleCNN()

# Loss function (cross-entropy for classification)
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# DataLoader for batching
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [5]:
# Training loop
num_epochs = 3  # You can adjust the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass

        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        # Track loss and accuracy
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct_preds / total_preds
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

Epoch 1/3, Loss: 0.4274, Accuracy: 0.8497
Epoch 2/3, Loss: 0.3366, Accuracy: 0.8841
Epoch 3/3, Loss: 0.3117, Accuracy: 0.8936


In [6]:
# Evaluate the model
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    correct_preds = (predicted == y_test).sum().item()
    accuracy = correct_preds / len(y_test)
    print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7287


In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'simple_cnn_model.pth')

# Load the model (if needed)
model = SimpleCNN()
model.load_state_dict(torch.load('simple_cnn_model.pth'))
model.eval()





---


---



---
**Dataset PAMAP2**



---



---

In [ ]:
#remove columns heartrate and temp of the three IMUs

import pandas as pd

# Define the file path
train_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/pamap2.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(train_data_file)

# Remove columns
df = df.drop(df.columns[2], axis=1)
df = df.drop(df.columns[2], axis=1)
df = df.drop(df.columns[11], axis=1)
df = df.drop(df.columns[20], axis=1)


# Save the updated DataFrame to a new CSV file
output_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/pamap2_columns_removed.csv'
df.to_csv(output_file, index=False)

# Print a message to confirm that the file was saved
print(f"Updated CSV saved to: {output_file}")


In [ ]:
# Replace the activity names in the first column with their corresponding IDs

import pandas as pd

# Define the file path for the modified CSV
modified_train_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/pamap2_columns_removed.csv'

# Load the modified CSV into a DataFrame without assuming a header
df = pd.read_csv(modified_train_data_file, header=None)

# Print the first few rows to check the content
print("Original DataFrame first few rows:")
print(df.head())

# Create a dictionary to map activity names to activity IDs
activity_map = {
    'lying': 1,
    'sitting': 2,
    'standing': 3,
    'walking': 4,
    'running': 5,
    'cycling': 6,
    'nordic_walking': 7,
    'watching_TV': 9,
    'computer_work': 10,
    'car_driving': 11,
    'ascending_stairs': 12,
    'descending_stairs': 13,
    'vacuum_cleaning': 16,
    'ironing': 17,
    'folding_laundry': 18,
    'house_cleaning': 19,
    'playing_soccer': 20,
    'rope_jumping': 24,
    'other': 0
}

# Replace the activity names in the first column with their corresponding IDs
df.iloc[:, 0] = df.iloc[:, 0].map(activity_map)

# Print the first few rows after the transformation to ensure it's working
print("\nDataFrame after replacing activity names with IDs:")
print(df.head())

# Save the updated DataFrame to a new CSV file
output_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/pamap2_with_activityIDs.csv'
df.to_csv(output_file, index=False, header=False)

# Print a confirmation message
print(f"Updated CSV with activity IDs saved to: {output_file}")


In [ ]:
#Split CSV in train (excluding user 5) and test (only user 5) csv files

import pandas as pd

# Define the file path for the CSV file containing activity IDs
train_data_file_with_ids = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/pamap2_with_activityIDs.csv'

# Load the CSV file into a DataFrame without assuming a header
df = pd.read_csv(train_data_file_with_ids, header=None)

# Print the first few rows to check the content
print("Original DataFrame first few rows:")
print(df.head())

# Filter rows where the user ID is 5 and save them as test_pamap2.csv
test_df = df[df.iloc[:, 1] == 5]
test_df.to_csv('/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/test_pamap2.csv', index=False, header=False)

# Filter rows where the user ID is not 5 (excluding) and save them as train_pamap2.csv
train_df = df[df.iloc[:, 1] != 5]
train_df.to_csv('/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/train_pamap2.csv', index=False, header=False)

# Print a confirmation message
print("Data has been split into train_pamap2.csv (excluding user ID 5) and test_pamap2.csv (user ID 5 only).")


In [ ]:
#Convert .csv to .dat

import pandas as pd

# Define file paths
train_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/train_pamap2.csv'
test_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/test_pamap2.csv'

# Function to convert rows into the required format and save as .dat
def convert_to_dat(input_file, output_file):
    # Load the CSV file without a header
    df = pd.read_csv(input_file, header=None)

    # Open the output .dat file to write
    with open(output_file, 'w') as file:
        for _, row in df.iterrows():
            # Convert row into the required format: {{column 2, column 3, column 4, ...}, column 0}
            row_data = "{{" + ",".join(map(str, row[2:])) + "}," + str(int(row[0])) + "}\n"
            file.write(row_data)

# Convert train_pamap2.csv to train_pamap2.dat
convert_to_dat(train_file, '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/train_pamap2.dat')

# Convert test_pamap2.csv to test_pamap2.dat
convert_to_dat(test_file, '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/test_pamap2.dat')

print("Conversion to .dat files is complete!")


Conversion to .dat files is complete!


In [ ]:
# Create time windows

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
import re

# Function to create time windows
def create_time_windows(data, labels, window_size):
    X = []
    y = []

    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])  # Select window of data
        y.append(labels[i + (window_size-1)])  # Label is from the last element of the window

    return np.array(X), np.array(y)

# Updated data loading function
def load_data(file_path):
    features = []
    labels = []

    with open(file_path, 'r') as f:
        for line in f:
            # Use regular expression to match the pattern {{feature_vector}, label}
            match = re.match(r"\{\{([0-9.,-]+)\},\s*(\d+)\}", line.strip())

            if match:
                # Extract feature vector and label
                feature_str = match.group(1)  # The feature string "8.24,-2.11,3.87"
                label = int(match.group(2))  # The label "4"

                # Convert the feature string to a list of floats
                feature_vector = list(map(float, feature_str.split(',')))

                features.append(feature_vector)
                labels.append(label)

    return np.array(features), np.array(labels)

# Load train and test data
train_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/train_pamap2.dat'  # Adjust path to your file
test_data_file = '/content/drive/MyDrive/MyBox/prof/projetos de pesquisa/pauNaJaca/dataset/PAMAP2/test_pamap2.dat'  # Adjust path to your file

print("File loaded:")

# Define the window size
window_size = 10

# Create time windows
X_train, y_train = load_data(train_data_file)
X_train, y_train = create_time_windows(X_train, y_train, window_size)

X_test, y_test = load_data(test_data_file)
X_test, y_test = create_time_windows(X_test, y_test, window_size)

print("Time window created:")

# Print the first feature vector and label
print("First feature vector in X_train:")
print(X_train[0])  # First row (first feature vector)
print("Second feature vector in X_train:")
print(X_train[1])  # First row (first feature vector)

print("First label in y_train:")
print(y_train[0])  # First label
print("Second label in y_train:")
print(y_train[1])  # First label

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # For classification (long type for labels)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# # Print the first feature vector and label
# print("First feature vector in X_train:")
# print(X_train[0])  # First row (first feature vector)
# print("First label in y_train:")
# print(y_train[0])  # First label

# Check the shapes of the loaded data
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
# Define a model

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # CNN layers
        self.conv1 = nn.Conv1d(in_channels=27, out_channels=27, kernel_size=3, padding=1)  # input channels = number of features
        self.conv2 = nn.Conv1d(in_channels=27, out_channels=16, kernel_size=3, padding=1)  # input channels = number of features

        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        # Fully connected layers
        # Calculate the input size for the fully connected layer based on the output size of conv1
        self.fc1 = nn.Linear(16 * (X_train.shape[1] // 2), 64)  # Flattened size after pooling
        self.fc2 = nn.Linear(64, 25)  # Assuming 25 classes for classification

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Add channel dimension (change shape to [batch_size, channels, seq_length])

        x = torch.relu(self.conv1(x))  # Apply conv1 and pooling
        x = self.pool(torch.relu(self.conv2(x)))  # Apply conv2 and pooling
        #x = self.pool(torch.relu(self.conv1(x)))  # Apply conv1 and pooling

        x = x.view(-1, 16 * (x.shape[2]))  # Flatten for fully connected layer
        x = torch.relu(self.fc1(x))  # Apply first fully connected layer
        x = self.fc2(x)  # Output layer (no activation since we'll apply softmax in loss)

        return x


# Initialize the model
model = SimpleCNN()

# Loss function (cross-entropy for classification)
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# DataLoader for batching
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# Training loop

num_epochs = 3  # You can adjust the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass

        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        # Track loss and accuracy
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct_preds / total_preds
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

Epoch 1/3, Loss: 0.2979, Accuracy: 0.9041
Epoch 2/3, Loss: 0.1936, Accuracy: 0.9378
Epoch 3/3, Loss: 0.1731, Accuracy: 0.9447


In [ ]:
# Evaluate the model
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    correct_preds = (predicted == y_test).sum().item()
    accuracy = correct_preds / len(y_test)
    print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7133
